In [12]:
# 크롤링
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
driver = webdriver.Chrome('C:/chromedriver.exe')
for i in range(4):
    
    # 1982년 부터 2018년 까지 statiz에 기록된 선수들 필터링 (총 8558명)
    url = 'http://www.statiz.co.kr/stat.php?mid=stat&re=0&ys=2019&ye=2019&sn=100&pa={}'.format(i*100)
    
    driver.get(url)
    driver.implicitly_wait(5)
    
    html = driver.find_element_by_xpath('//*[@id="mytable"]/tbody').get_attribute("innerHTML") #기록 table을 str형태로 저장
    soup = BeautifulSoup(html, 'html.parser') #str 객체를 BeautifulSoup 객체로 변경
    
    temp = [i.text.strip() for i in soup.findAll("tr")] #tr 태그에서, text만 저장하기
    temp = pd.Series(temp) #list 객체에서 series 객체로 변경
    
    # 즉, 선수별 기록만 남기고, index를 reset 해주기
    temp = temp[~temp.str.match("[순W]")].reset_index(drop=True) 
    temp = temp.apply(lambda x: pd.Series(x.split(' '))) #띄어쓰기 기준으로 나눠서 dataframe으로 변경
    
    #선수 팀 정보 이후 첫번째 기록과는 space 하나로 구분, 그 이후로는 space 두개로 구분이 되어 있음 
    #그래서 space 하나로 구분을 시키면, 빈 column들이 존재 하는데, 해당 column들 제거 
    temp = temp.replace('', np.nan).dropna(axis=1) 
    
    #WAR 정보가 들어간 column이 2개 있다. (index가 1인 column과, 제일 마지막 column)
    #그 중에서 index가 1인 columm 제거 
    #temp = temp.drop(1, axis=1)
    
    #선수 이름 앞의 숫자 제거
    temp[0] = temp[0].str.replace("^\d+", '')
    # 선수들의 생일 정보가 담긴 tag들 가지고 오기
    #birth = [i.find("a") for i in soup.findAll('tr') if 'birth' in i.find('a').attrs['href']]
    
    # tag내에서, 생일 날짜만 추출하기 
    #p = re.compile("\d{4}\-\d{2}\-\d{2}")
    #birth = [p.findall(i.attrs['href'])[0] for i in birth]
    
    # 생일 column 추가
    #temp['생일'] = birth
    
    # page별 완성된 dataframe을 계속해서 result에 추가 시켜주기 
    if i == 0:
        result = temp
    else:
        result = result.append(temp)
        result = result.reset_index(drop=True)
        
    print(i, "완료")
        
#column 명 정보 저장        
columns = ['선수'] + [i.text for i in soup.findAll("tr")[0].findAll("th")][4:-3] + ['타율', '출루', '장타', 'OPS', 'wOBA', 'wRC+', 'WAR+', 'WPA']

#column 명 추가
result = result.drop(columns=1)
result.columns = columns

#webdriver 종료
driver.close()

print("최종 완료")

result.info()

0 완료
1 완료
2 완료
3 완료
최종 완료
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356 entries, 0 to 355
Data columns (total 28 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   선수      356 non-null    object
 1   G       356 non-null    object
 2   타석      356 non-null    object
 3   타수      356 non-null    object
 4   득점      356 non-null    object
 5   안타      356 non-null    object
 6   2타      356 non-null    object
 7   3타      356 non-null    object
 8   홈런      356 non-null    object
 9   루타      356 non-null    object
 10  타점      356 non-null    object
 11  도루      356 non-null    object
 12  도실      356 non-null    object
 13  볼넷      356 non-null    object
 14  사구      356 non-null    object
 15  고4      356 non-null    object
 16  삼진      356 non-null    object
 17  병살      356 non-null    object
 18  희타      356 non-null    object
 19  희비      356 non-null    object
 20  타율      156 non-null    object
 21  출루      156 non-null    object
 22  

KeyError: 300

In [ ]:
from selenium import webdriver
import urllib.request
import urllib.parse
import pandas as pd
import re


def get_url(url, enc='utf-8'):
    req = urllib.request.Request(url)
    response = urllib.request.urlopen(req)

    if response.getcode() == 200:
        data = response.read()

        return data
    else:
        return None

def get_crawl(year, teamName):

    playerStat = []

    url = "http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys=%s&ye=%s" \
          "&se=0&te=%s&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=2&o1=FIP" \
          "&de=0&o2=WAR&lr=0&tr=&cv=&ml=1&sn=30&si=&cn="\
          % (year, year, urllib.parse.quote_plus(teamName))

    browser = webdriver.Chrome('C:\Chromedriver.exe')
    browser.implicitly_wait(2)

    browser.get(url)

    selectedTr = browser.find_elements_by_css_selector('table#mytable tr')[2:]

    for index, tr in enumerate(selectedTr):
        tdList = tr.text.split(' ')
        # print(tdList)
        if (ord("0") > ord(tdList[0][0])) or (ord("9") < ord(tdList[0][0])) :
            continue

        for td in tdList:
            if td == '':
                td = str(0)

        playerStat.append(tdList[0:17])

    print(playerStat)

    browser.close()
    return playerStat

def main():
    yearScope = range(2019, 2020)
    teamNameList = ['두산','삼성','NC','넥센','SK','한화','LG','롯데','KIA','kt', '히어로즈']

    columnList = ['순', '이름', '팀', 'FIP', '출장', '이닝', 'ERA', 'FIP',
                  'K/9', 'BB/9', 'HR/9', 'K/BB', 'PFR', 'BIPA', 'LOB%',
                  '타율', '출루율']

    for year in yearScope:
        for teamName in teamNameList:
            recordList = get_crawl(year, teamName)
            if len(recordList) != 0:
                # print(recordList)
                recordTable = pd.DataFrame(recordList, columns=columnList)
                recordTable.to_csv(str(year) + str(teamName) + '_pitcher.csv', encoding="utf-8", mode="w")

                del recordTable
            del recordList

if __name__ == "__main__":
    main()